In [13]:
#Task 3 April Tags

import cv2
import glob
import pyapriltags
import numpy as np

#NOTE:
#using april tag use actual april tags // use aruco tag detector instead (aruco and open cv) ; 
#both have dictionaries 3x3 4x4 5x5 (one of the paremeters i need to pass to the detector)
#better to create the tag myself and know what dictionary is being used. 

# Load all calibration images from a folder
image_files = glob.glob('Task3_april_calib/*.jpg')
images = [cv2.imread(fname) for fname in image_files]

# Initialize the detector
detector = pyapriltags.Detector(families='tag36h11')
# Use the correct tag family for your AprilTags
# For example, if your tags are from the 'tag25h9' family, use:
# detector = pyapriltags.Detector(families='tag25h9')
# Replace 'tag25h9' with the actual family your tags belong to.
# Store detected corners and IDs
all_corners = []
all_ids = []

for img in images:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    results = detector.detect(gray)
    corners = [r.corners for r in results]
    ids = [r.tag_id for r in results]
    all_corners.append(corners)
    all_ids.append(ids)

# Define the grid size and tag spacing (in meters)
grid_rows = 1
grid_cols = 1
tag_size = 0.15  # 15 cm

# Generate 3D object points for each tag
object_points = []
for row in range(grid_rows):
    for col in range(grid_cols):
        # Each tag's center
        object_points.append([col * tag_size, row * tag_size, 0])
object_points = np.array(object_points, dtype=np.float32)

image_points = []
object_points_list = []

for corners, ids in zip(all_corners, all_ids):
    img_pts = []
    obj_pts = []
    for i, tag_id in enumerate(ids):
        img_pts.extend(corners[i])  # 4 corners per tag
        # Map tag_id to object_points index
        obj_pts.extend([object_points[tag_id]] * 4)
    image_points.append(np.array(img_pts, dtype=np.float32))
    object_points_list.append(np.array(obj_pts, dtype=np.float32))

# Filter out empty sets
filtered_image_points = []
filtered_object_points_list = []

for img_pts, obj_pts in zip(image_points, object_points_list):
    if len(img_pts) > 0 and len(obj_pts) > 0:
        filtered_image_points.append(img_pts)
        filtered_object_points_list.append(obj_pts)

print("Number of valid image sets:", len(filtered_image_points))
for i, (obj_pts, img_pts) in enumerate(zip(filtered_object_points_list, filtered_image_points)):
    print(f"Image {i}: object_points shape = {obj_pts.shape}, image_points shape = {img_pts.shape}")

# Only use filtered lists for calibration
if len(filtered_image_points) > 0 and len(filtered_object_points_list) > 0:
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        filtered_object_points_list, filtered_image_points, images[0].shape[1::-1], None, None
    )
    print("Camera matrix:\n", camera_matrix)
    print("Distortion coefficients:\n", dist_coeffs)
else:
    print("No valid AprilTag detections found. Camera calibration cannot proceed.")


#source
#1. https://github.com/AprilRobotics/apriltag
#2. https://github.com/abhishekpadalkar/camera_calibration/blob/master/camera_calibration.py


Number of valid image sets: 11
Image 0: object_points shape = (4, 3), image_points shape = (4, 2)
Image 1: object_points shape = (4, 3), image_points shape = (4, 2)
Image 2: object_points shape = (4, 3), image_points shape = (4, 2)
Image 3: object_points shape = (4, 3), image_points shape = (4, 2)
Image 4: object_points shape = (4, 3), image_points shape = (4, 2)
Image 5: object_points shape = (4, 3), image_points shape = (4, 2)
Image 6: object_points shape = (4, 3), image_points shape = (4, 2)
Image 7: object_points shape = (4, 3), image_points shape = (4, 2)
Image 8: object_points shape = (4, 3), image_points shape = (4, 2)
Image 9: object_points shape = (4, 3), image_points shape = (4, 2)
Image 10: object_points shape = (4, 3), image_points shape = (4, 2)


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\calibration.cpp:94: error: (-215:Assertion failed) matH0.size() == Size(3, 3) in function 'cv::initIntrinsicParams2D'
